In [1]:
from auxiliary import loadData_staticTargetAddrMatch, prepare_data_loaders
from algorithms import RssPosAlgo_NeuralNet_MLP4layer
from algorithms import RssPosAlgo_NeuralNet_supervisedTrainingLoop
from algorithms import RssPosAlgo_NearestNeighbour_Interpolation
from algorithms import RssPosAlgo_NearestNeighbour_GetKmeansDb
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

### prepare model, dataloaders and training parameters

In [4]:
datajsonpath = "../experiments/exp004_20241022_sna_kadirburakerdem/data-tshark/data.json"
inp_rss_vals, gt_locations = loadData_staticTargetAddrMatch(datajsonpath, second_hold = 5, shuffle=False, 
                                                            target_addresses=["d8:47:32:eb:6c:38",
"50:c7:bf:19:e6:4d",
"4c:77:6d:5f:dc:20"], snap250ms=False)

epochs           = 601
batch_size       = 8
train_test_split = 0.8
window_size = 50

train_loader, test_loader, xtr, ytr, xts, yts = prepare_data_loaders(inp_rss_vals, gt_locations, 
                                                                     batch_size = 32, 
                                                                     train_test_split = train_test_split)

print("Subset sizes | train:", xtr.shape[0], ", test:",xts.shape[0])
MLP = RssPosAlgo_NeuralNet_MLP4layer(window_size = window_size, inch=3)
MLP.train()

MLP_criterion = nn.MSELoss(reduction='mean')
MLP_optimizer = optim.Adam(MLP.parameters(), lr=3e-4)

Subset sizes | train: 4938 , test: 1235


### train model

In [5]:
MLP = RssPosAlgo_NeuralNet_supervisedTrainingLoop(train_loader = train_loader, 
                                                         test_loader  = test_loader,
                                                         model        = MLP, 
                                                         criterion    = MLP_criterion, 
                                                         optimizer    = MLP_optimizer, 
                                                         epochs       = epochs,
                                                         testfreq     = 20) # testfreq is in epochs

Epoch [1/601] test loss: 4.874, training loss: -1.000
Epoch [21/601] test loss: 1.441, training loss: 1.254
Epoch [41/601] test loss: 1.429, training loss: 1.250
Epoch [61/601] test loss: 1.477, training loss: 1.249
Epoch [81/601] test loss: 1.489, training loss: 1.241
Epoch [101/601] test loss: 1.363, training loss: 1.241
Epoch [121/601] test loss: 1.242, training loss: 1.232
Epoch [141/601] test loss: 1.329, training loss: 1.229
Epoch [161/601] test loss: 1.442, training loss: 1.229
Epoch [181/601] test loss: 1.312, training loss: 1.229
Epoch [201/601] test loss: 1.590, training loss: 1.225
Epoch [221/601] test loss: 1.352, training loss: 1.227
Epoch [241/601] test loss: 1.397, training loss: 1.220
Epoch [261/601] test loss: 1.359, training loss: 1.221
Epoch [281/601] test loss: 1.444, training loss: 1.219
Epoch [301/601] test loss: 1.486, training loss: 1.215
Epoch [321/601] test loss: 1.377, training loss: 1.209
Epoch [341/601] test loss: 1.415, training loss: 1.205
Epoch [361/601]

### save the model

In [6]:
torch.save(MLP.state_dict(), 'savedmodels/dev008_exp004_MLP4layer.pth')

### evaluate model

In [7]:
MLP = RssPosAlgo_NeuralNet_MLP4layer(inch=3)
MLP.load_state_dict(torch.load('savedmodels/dev008_exp004_MLP4layer.pth'));
MLP.eval();

### Evaluate model with window_size sample at each time

RssPosAlgo_NearestNeighbour_Interpolation is discarded. Instead of MSE, euclidian distance have been utilized for error calculation.

In [41]:
error = 0  # Initialize the error accumulator

for test_idx in range(1, (len(xts) // window_size) + 1):
    start_idx = (test_idx - 1) * window_size
    end_idx = test_idx * window_size
    x_test_sample = xts[start_idx:end_idx]
    
    # Pass the sample through the MLP model to get predictions
    loc_pred_mlp = MLP(x_test_sample)
    
    # Ensure both tensors are detached and converted to numpy arrays for calculation
    loc_pred_mlp_np = loc_pred_mlp.detach().numpy()  # Predicted values
    yts_batch_np = yts[start_idx:end_idx].detach().numpy()  # True values

    diff = yts_batch_np - loc_pred_mlp_np
    # Calculate the squared error for the batch
    error1 = 0
    for i in range(len(diff)):
        error1 += np.linalg.norm(diff[i])
    error += error1/window_size
# Calculate the mean squared error over all batches
print("Mean error:", error/test_idx)

Mean error: 1.591053332090378


### Evaluate model with 1 sample at each time

In [12]:
db_kmeans = RssPosAlgo_NearestNeighbour_GetKmeansDb(xtr, ytr, num_clusters=3)
meanerror_nene_interp = 0;
meanerror_mlp         = 0;
for test_idx, x_test_sample in enumerate(xts): 
    loc_pred_mlp           = MLP(x_test_sample)
    loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(x_test_sample, db_kmeans)
    meanerror_mlp         += np.linalg.norm(yts[test_idx].numpy() - loc_pred_mlp.detach().numpy())
    meanerror_nene_interp += np.linalg.norm((yts[test_idx].numpy() - loc_pred_nene_interp))
print("Mean Error        :", np.linalg.norm(meanerror_mlp/(test_idx+1)))
print("NeNe+Interp:", np.linalg.norm(meanerror_nene_interp/(test_idx+1)))



Mean Error        : 1.5666769655006618
NeNe+Interp: 1.6491749335762056


### test a few samples

from test set

In [13]:
sampleid = 10
loc_pred_mlp           = MLP(xts[sampleid])
loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(xts[sampleid], db_kmeans)
meanerror_mlp         += (yts[sampleid].numpy() - loc_pred_mlp.detach().numpy())**2
meanerror_nene_interp += (yts[sampleid].numpy() - loc_pred_nene_interp)**2
print("Actual position  :", yts[sampleid].numpy())
print("MLP prediction   :", loc_pred_mlp.detach().numpy())
print("NeNe+Interp pred :", loc_pred_nene_interp)

Actual position  : [0.41500485 0.52060187]
MLP prediction   : [1.5077429 2.1150782]
NeNe+Interp pred : (1.8124692335993735, 2.3469841814898755)


---

from train set

In [14]:
sampleid = 20
loc_pred_mlp           = MLP(xtr[sampleid])
loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(xtr[sampleid], db_kmeans)
meanerror_mlp         += (ytr[sampleid].numpy() - loc_pred_mlp.detach().numpy())**2
meanerror_nene_interp += (ytr[sampleid].numpy() - loc_pred_nene_interp)**2
print("Actual position  :", ytr[sampleid].numpy())
print("MLP prediction   :", loc_pred_mlp.detach().numpy())
print("NeNe+Interp pred :", loc_pred_nene_interp)

Actual position  : [1.7463105 1.2628807]
MLP prediction   : [2.1316237 2.0416436]
NeNe+Interp pred : (2.1297793363800515, 2.0701069561292242)
